# Fitbit Foodlog + Activity
## From fitbit API
### Setup the basics

Import the api dataset and generate pandas dataframes for each set of data

In [19]:
# Import Modules
import os, json
import pandas as pd
import pdfkit as pdf
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import warnings

# Global Settings
root_folder = './dataset/api'
warnings.filterwarnings('ignore')

# Load Data
df_prof = pd.read_json(f"{root_folder}/profile.json")
df_nutri = pd.read_json(f"{root_folder}/nutrition.json")
# df_weight = pd.read_json(f"{root_folder}/weight.json")
# df_bfat = pd.read_json(f"{root_folder}/bodyfat.json")

# Function for displaying a bar graph
def bar_graph(df,xaxis,yaxis,title,figsize):
  df = df.groupby(xaxis, as_index=False).agg({yaxis: 'sum'})
  plt.bar(df[xaxis], df[yaxis])
  plt.title(title)
  plt.xlabel(xaxis)
  plt.ylabel(yaxis)
  plt.xticks(rotation=80)
  plt.grid(True)
  return plt

meal_type_mapping = {
    1: 'Breakfast',
    2: 'Morning Snack',
    3: 'Lunch',
    4: 'Afternoon Snack',
    5: 'Dinner',
    6: 'Evening Snack'
}


### Produce Nutrition Report

In [26]:
df_nutri_list = []
for index in df_nutri:
  df_norm = pd.json_normalize(df_nutri[index].foods)
  df = df_norm[[
    'logDate',
    'loggedFood.mealTypeId',
    'loggedFood.brand',
    'loggedFood.name',
    'loggedFood.amount',
    'loggedFood.unit.name',
    'nutritionalValues.calories',
    'nutritionalValues.carbs',
    'nutritionalValues.fat',
    'nutritionalValues.protein'
    ]]
  df.columns = ['Date','Meal','Brand','Name','Amount','Unit','Calories','Carbs','Fat','Protein']
  df['Meal'] = df['Meal'].map(meal_type_mapping)
  df.loc["Total"] = df.sum(numeric_only=True).rename('Total')
  df_disp = df.style.format({'Amount':'{:.0f}', 'Calories':'{:.0f}', 'Carbs': '{:.2f}', 'Fat': '{:.2f}', 'Protein': '{:.2f}' })
  df_nutri_list.append(df_disp)

html_list = []
for df in df_nutri_list:
    dfhtml = df.to_html()
    html_list.append(dfhtml)


html = """
<html>
  <head>
    <style> 
      body {font-family: Arial, sans-serif;}
      th, td {text-align: justify;}
    </style>
  </head>
<body>
  <h1>Report on Nutrition</h1>
"""
html += '<br />'.join(html_list)
html += '</body></html>'

html_path = f'{root_folder}/nutrition.html'
with open(html_path, 'w') as f:
    f.write(html)

pdffile = f'{root_folder}/nutrition.pdf'
pdfopt = { 'page-size': 'A4', 'orientation': 'Landscape' }
pdf.from_file(html_path, pdffile, options=pdfopt)


True

### Calories per day

In [21]:
# # Plotting the graph
# bar_graph(df_weight,'Date','Weight','Weight Graph'(10,6))

# # Display the plot
# plt.show()
